In [1]:
import warnings 
warnings.simplefilter('ignore')

import scanpy as sc
import scparadise
import pandas as pd
import numpy as np
import pandas as pd
import os
import scvi
import torch

In [2]:
os.chdir('/mnt/c/Users/vadim/scRNA/scParadise/scripts_article')

In [3]:
adata = sc.read_h5ad('Mouse_aging_brain/adata.h5ad')

In [4]:
# Create adata_train and test datasets for model training
adata_train = adata[adata.obs['orig.ident'].isin(['young2', 'old1', 'oldex1'])].copy()
adata_test_1 = adata[adata.obs['orig.ident'].isin(['old2'])].copy()
adata_test_2 = adata[adata.obs['orig.ident'].isin(['old4'])].copy()
adata_test_3 = adata[adata.obs['orig.ident'].isin(['oldex2'])].copy()
adata_test_4 = adata[adata.obs['orig.ident'].isin(['oldex4'])].copy()
adata_test_5 = adata[adata.obs['orig.ident'].isin(['young1'])].copy()
adata_test_6 = adata[adata.obs['orig.ident'].isin(['young4'])].copy()
for i in [adata_train, adata_test_1, adata_test_2, adata_test_3, adata_test_4, adata_test_5, adata_test_6]:
    i.layers['counts'] = i.X.copy()
    sc.pp.normalize_total(i, target_sum=None)
    sc.pp.log1p(i)
    i.raw = i

# Subset genes
genes = pd.read_csv('Mouse_aging_brain/genes_for_AI.csv')
adata_train = adata_train[:, genes.genes].copy()

In [5]:
scvi.model.SCVI.setup_anndata(adata_train, batch_key="orig.ident", layer="counts")

In [20]:
scvi_ref = scvi.model.SCVI(
    adata_train
)
scvi_ref.train(train_size=0.89) # default train_size = 0.9 leads to failing of model training

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 400/400: 100%|██████████████| 400/400 [19:31<00:00,  2.84s/it, v_num=1, train_loss_step=287, train_loss_epoch=283]

`Trainer.fit` stopped: `max_epochs=400` reached.


Epoch 400/400: 100%|██████████████| 400/400 [19:31<00:00,  2.93s/it, v_num=1, train_loss_step=287, train_loss_epoch=283]


In [21]:
scvi_ref_path = 'Mouse_aging_brain/scanvi_test/models'
scvi_ref.save(scvi_ref_path, overwrite=True)

In [23]:
scanvi_ref = scvi.model.SCANVI.from_scvi_model(
    scvi_ref,
    unlabeled_category="Unknown",
    labels_key='Celltype',
)
# Train scanvi models for selected annotation level
scanvi_ref.train(train_size=0.89) # default train_size = 0.9 leads to failing of model training
scanvi_ref_path = os.path.join('Mouse_aging_brain/scanvi_test/models', 'Celltype')
scanvi_ref.save(scanvi_ref_path, overwrite=True)

INFO     Training for 10 epochs.                                                                                   


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 10/10: 100%|██████████████████| 10/10 [00:53<00:00,  5.28s/it, v_num=1, train_loss_step=270, train_loss_epoch=267]

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 10/10: 100%|██████████████████| 10/10 [00:53<00:00,  5.35s/it, v_num=1, train_loss_step=270, train_loss_epoch=267]


In [24]:
# Create list of test datasets
lst_test_folders = ['old2', 'old4', 'oldex2', 'oldex4', 'young1', 'young4']
lst_test_adatas = [adata_test_1, adata_test_2, adata_test_3, adata_test_4, adata_test_5, adata_test_6]

In [26]:
for adata_test, folder in zip(lst_test_adatas, lst_test_folders):
    os.makedirs(os.path.join('Mouse_aging_brain/scanvi_test/reports', folder))
    # Train scanvi model for test dataset (adata_test)
    scvi.model.SCANVI.prepare_query_anndata(adata_test, scanvi_ref)
    scanvi_test = scvi.model.SCANVI.load_query_data(adata_test, scanvi_ref)
    scanvi_test.train(
        max_epochs=10
    )
    # Create name for predicted annotation level
    SCANVI_annotation = 'scanvi_Celltype'
    # Predict annotation using pretrained scanvi_test model
    adata_test.obs[SCANVI_annotation] = scanvi_test.predict()
    # Create and save classification report of selected annotation level
    file_save = 'report_test_scanvi_default_' + 'Celltype' + '.csv'
    scparadise.scnoah.report_classif_full(adata_test, 
                                          celltype = 'Celltype', 
                                          pred_celltype = SCANVI_annotation, 
                                          report_name = file_save,
                                          save_path = os.path.join('Mouse_aging_brain/scanvi_test/reports', folder),
                                          save_report=True)

INFO     Found 100.0% reference vars in query data.                                                                
INFO     Training for 10 epochs.                                                                                   


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 10/10: 100%|██████████████████| 10/10 [00:14<00:00,  1.50s/it, v_num=1, train_loss_step=267, train_loss_epoch=270]

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 10/10: 100%|██████████████████| 10/10 [00:14<00:00,  1.49s/it, v_num=1, train_loss_step=267, train_loss_epoch=270]
Successfully saved report

INFO     Found 100.0% reference vars in query data.                                                                
INFO     Training for 10 epochs.                                                                                   


...

Epoch 10/10: 100%|██████████████████| 10/10 [00:18<00:00,  1.90s/it, v_num=1, train_loss_step=279, train_loss_epoch=287]
Successfully saved report

INFO     Found 100.0% reference vars in query data.                                                                
INFO     Training for 10 epochs.                                                                                   


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 10/10: 100%|██████████████████| 10/10 [00:41<00:00,  4.22s/it, v_num=1, train_loss_step=294, train_loss_epoch=301]

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 10/10: 100%|██████████████████| 10/10 [00:41<00:00,  4.17s/it, v_num=1, train_loss_step=294, train_loss_epoch=301]
Successfully saved report

